In [238]:
X = torch.Tensor{{1,0}, {1,1}, {0,1}}
Y = torch.Tensor{{1,0,0}, {0,1,0}, {0,0,1}}
print(X)
print(Y)

cc = torch.zeros(1, Y:size(1))
fc = torch.zeros(Y:size(1), X:size(2))
lp = torch.zeros(Y:size(1), X:size(2))
print(cc)
print(fc)

 1  0
 1  1
 0  1
[torch.DoubleTensor of size 3x2]

 1  0  0
 0  1  0
 0  0  1
[torch.DoubleTensor of size 3x3]

 0  0  0
[torch.DoubleTensor of size 1x3]

 0  0
 0  0
 0  0
[torch.DoubleTensor of size 3x2]



In [239]:
function counts(X, Y)
    fc:add(Y:t()*(X))
    cc:add(Y:sum(1))
end

In [240]:
function update_feature_log_prob(fc)
    smoothed_fc = torch.add(fc, 1)
    smoothed_cc = smoothed_fc:sum(2)
    ones = torch.ones(smoothed_cc:size(1), smoothed_fc:size(2))
    print(ones)
    print(smoothed_cc)
    torch.addcmul(ones, smoothed_cc)
end   

In [241]:
counts(X,Y)
update_feature_log_prob(fc)

invalid arguments: DoubleTensor DoubleTensor 
expected arguments: [*DoubleTensor*] DoubleTensor [double] DoubleTensor DoubleTensor
stack traceback:
	[C]: at 0x04dc7f00
	[C]: in function 'addcmul'
	[string "function update_feature_log_prob(fc)..."]:7: in function 'update_feature_log_prob'
	[string "counts(X,Y)..."]:2: in main chunk
	[C]: in function 'xpcall'
	/Users/Vincent/torch/install/share/lua/5.1/itorch/main.lua:179: in function </Users/Vincent/torch/install/share/lua/5.1/itorch/main.lua:143>
	/Users/Vincent/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	...s/Vincent/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	...s/Vincent/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	...s/Vincent/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	/Users/Vincent/torch/install/share/lua/5.1/itorch/main.lua:350: in main chunk
	[C]: in function 'require'
	(command line):1: in main chunk
	[C]: at 0x0104

 1  1
 1  1
 1  1
[torch.DoubleTensor of size 3x2]

 3
 4
 3
[torch.DoubleTensor of size 3x1]



In [188]:
torch.ones(3, 2)*

 1  1
 1  1
 1  1
[torch.DoubleTensor of size 3x2]

